### Step-1 Import Modules and  prepare the training data se

In [ ]:
import numpy as np
# import tensorflow as tf
from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler


# prepare the data set to be trained
train_labels = []
train_samples = []

for i in range(50):
    #5%  of young experieance side effect
    random_younger = randint(13,64)
    train_samples.append(random_younger)
    train_labels.append(1)

    # 5% of older do not experiance side effect
    random_older = randint(64,100)
    train_samples.append(random_older)
    train_labels.append(0)

for i in range(1000):
    #5%  of young do not experieance side effect
    random_younger = randint(13,64)
    train_samples.append(random_younger)
    train_labels.append(0)

    # 95% of older experiance side effect
    random_older = randint(64,100)
    train_samples.append(random_older)
    train_labels.append(1)

# convert it to numpy array
train_labels = np.array(train_labels)
train_samples = np.array(train_samples)

#shuffle or randomize their order 
train_labels, train_samples = shuffle(train_labels,train_samples)

# scaling range, the default range is 0,1 => MinMaxScaler(feature_range=(0,1)) = MinMaxScaler()
scaler = MinMaxScaler(feature_range=(0,1))


scaled_train_samples = scaler.fit_transform(train_samples.reshape(-1,1))

### Step-2 Create Artificial Neural Network

In [ ]:
import tensorflow as tf
import keras as kr
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

# Available gpu devices
# physica_device = tf.config.experimental.list_physical_devices('GPU')
# print("Nums GPU Availabel : ", len(physica_device))

model = Sequential([
    Dense(units=16, input_shape=(1,), activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=2, activation='softmax')

])

model.summary()


### Step-3 Train the Artificial Neural Network with the dataset we have created above

In [ ]:
#prepate the model for traininig by compiling
model.compile(optimizer=Adam(learning_rate=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

#training occurs whenever we call fit function
model.fit(x=scaled_train_samples, y= train_labels,batch_size=10,epochs=30,shuffle=True,verbose=1)

<div> 

### Step-4 We can actually get an understanding of how welll our model is generalizeing via validation 
##### VAlidation set can be either 
###### 1, completely separate data or 
###### 2, take subset of trainig data before trainig begans and separte it in a separte set so that the trainig model can't access it
</div>

In [ ]:
#prepate the model for traininig by compiling
model.compile(optimizer=Adam(learning_rate=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

#training occurs whenever we call fit function
model.fit(x=scaled_train_samples, y= train_labels,validation_split=0.1, batch_size=10,epochs=30,shuffle=False,verbose=1)

### 

### Step-5 Creating a validation set for testing the performance of our model

In [ ]:
# creating testing data set
testing_samples = []
testing_labels = []

for i in range(50):
    #5%  of young experieance side effect
    random_younger = randint(13,64)
    testing_samples.append(random_younger)
    testing_labels.append(1)

    # 5% of older do not experiance side effect
    random_older = randint(64,100)
    testing_samples.append(random_older)
    testing_labels.append(0)

for i in range(1000):
    #5%  of young do not experieance side effect
    random_younger = randint(13,64)
    testing_samples.append(random_younger)
    testing_labels.append(0)

    # 95% of older experiance side effect
    random_older = randint(64,100)
    testing_samples.append(random_older)
    testing_labels.append(1)

# convert it to numpy array
testing_labels = np.array(testing_labels)
testing_samples = np.array(testing_samples)

#shuffle or randomize their order 
testing_labels, testing_samples = shuffle(testing_labels,testing_samples)
scaled_testing_samples = scaler.fit_transform(testing_labels.reshape(-1,1))

## Step-6 Use the test data 

In [ ]:
predictions = model.predict(x=scaled_testing_samples, batch_size=10, verbose=0)

rounded_predictions = np.argmax(predictions, axis=-1)

# print(predictions)
# print(rounded_predictions)


## Step-7 Confusion Matrix 
#### We need to use a Confusion Matrix to visualize the prediction result of neural net

In [ ]:
%matplotlib inline
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt



def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='confusion_matrix',
                          cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print('Normalized Confusion matrix ')
    else:
        print('Confusion Matrix, without Normalization')

    print(cm)
    thresh = cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment='center',
                 color='white' if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


cm = confusion_matrix(y_true=testing_labels, y_pred=rounded_predictions)
cm_plot_labels = ['no_side_effects', 'had_side_effects']
plot_confusion_matrix(cm=cm,classes=cm_plot_labels,title='confusion Matrix')
plt.show()

## Step-8 Save and Load a Model with Tensorflow Keras API

In [ ]:
model.summary()